In [1]:
import pkg_resources
from subprocess import call
import pandas as pd
import os

In [2]:
def command_to_file(command="pip3 list", file='lib.txt', delete=True):
    n = call(command + " > " + file, shell=True)
    f = open("lib.txt", "r")
    listeraw = [l[:-1] for l in f]
    columns = [v for v in listeraw[0].split(" ") if len(v) > 0][0:2]
    liste = [[v for v in l.split(" ") if len(v) > 0][0:2]
             for l in listeraw[2:]]
    if os.path.exists(file) and delete: os.remove(file)
    df = pd.DataFrame(liste, columns=columns)
    return df


def get_versions(file='lib.txt', delete=True):
    return command_to_file(command="pip3 list", file=file, delete=delete)


def get_outdated(file='lib.txt', delete=True):
    df_before = get_versions(file=file, delete=delete)
    df_outdated = command_to_file(command="pip3 list --outdated",
                                  file=file,
                                  delete=delete)
    df = df_before.merge(df_outdated,
                         how='left',
                         left_on='Package',
                         right_on='Package',
                         suffixes=('', '_outdated'))
    df['Version_outdated'] = ~df['Version_outdated'].isnull().values
    df.rename(columns={"Version_outdated": "Outdated"}, inplace=True)
    return df


def upgrade(df_in):
    if 'Outdated' in df_in.columns:
        packages = list(df_in.loc[df['Outdated'], 'Package'])
    else:
        packages = list(df_in['Package'])

    print("Number of upgrades :", len(packages))
    for package in packages:
        n = call("pip3 install --upgrade --user " + package, shell=True)
        print(n, package)  # 0 = ok

    df_after = get_versions()
    df_in = df_in.merge(df_after,
                        how='left',
                        left_on='Package',
                        right_on='Package',
                        suffixes=('_before', '_after'))
    df_in['Upgraded'] = df_in['Version_before'] != df_in['Version_after']
    return df_in

##### Get packages versions

In [3]:
df = get_versions()
print(df.to_string())

                               Package          Version
0                              absl-py            0.9.0
1                                 argh           0.26.2
2                           asn1crypto            1.3.0
3                                astor            0.8.1
4                              astroid            2.4.1
5                              astropy      4.0.1.post1
6                                 attr            0.3.1
7                                attrs           19.3.0
8                            audioread            2.1.8
9                             autopep8            1.5.2
10                            backcall            0.1.0
11                              bleach            3.1.5
12                                boto           2.49.0
13                               boto3           1.13.4
14                            botocore           1.16.4
15                              Brlapi            0.6.6
16                          cachetools          

##### Get outdated packages

In [4]:
df = get_outdated()
print(df.loc[df['Outdated']].to_string())

          Package Version  Outdated
33    distro-info   0.0.0      True
35       docutils  0.15.2      True
52   graphql-core   2.3.1      True
176         pyzmq  19.0.0      True
186            Rx   1.6.1      True
227         torch   1.4.0      True
248         wrapt  1.11.2      True


##### Update all packages
If 'Outdated' columns exists, upgrade only non up-to-date packages, else tries all.

In [5]:
df = upgrade(df)

Number of upgrades : 7
0 distro-info
0 docutils
0 graphql-core
0 pyzmq
0 Rx
0 torch
0 wrapt


##### See successful upgrades

In [6]:
if 'Outdated' in df.columns: print(df.loc[df['Outdated'] | df['Upgraded']].to_string())
else: print(df.loc[df['Upgraded']].to_string())

          Package Version_before  Outdated Version_after  Upgraded
33    distro-info          0.0.0      True         0.0.0     False
35       docutils         0.15.2      True          0.16      True
52   graphql-core          2.3.1      True         3.1.0      True
176         pyzmq         19.0.0      True        19.0.1      True
186            Rx          1.6.1      True         3.1.0      True
227         torch          1.4.0      True         1.5.0      True
248         wrapt         1.11.2      True        1.12.1      True


Sometimes some packages are downgraded -> re-upgrade  
Check outdated packages :

In [7]:
verif = get_outdated()
print(verif.loc[verif['Outdated']].to_string())

        Package Version  Outdated
33  distro-info   0.0.0      True


If a package failed to upgrade, manually upgrade it.  

In [8]:
print(df.to_string())

                               Package   Version_before  Outdated    Version_after  Upgraded
0                              absl-py            0.9.0     False            0.9.0     False
1                                 argh           0.26.2     False           0.26.2     False
2                           asn1crypto            1.3.0     False            1.3.0     False
3                                astor            0.8.1     False            0.8.1     False
4                              astroid            2.4.1     False            2.4.1     False
5                              astropy      4.0.1.post1     False      4.0.1.post1     False
6                                 attr            0.3.1     False            0.3.1     False
7                                attrs           19.3.0     False           19.3.0     False
8                            audioread            2.1.8     False            2.1.8     False
9                             autopep8            1.5.2     False     

##### Install a package

In [11]:
package = 'pandas'
call("pip3 install --upgrade --user " + package, shell=True)

0